# <a href="https://mipt-stats.gitlab.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Задание 5

**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`.
* Дедлайн **10 апреля в 23:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb` и все фотографии, если пишете теоретическую часть от руки.
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.

-----

*Замечания.* Теоретические решения можно оформить
* в $\LaTeX$-формате в ноутбуке;
* написать от руки и прикрепить к ноутбуку;
* написать от руки и выслать боту.  

Во втором случае также **важно** "вшить" фото в ноутбук. Сделать это можно с помощью Edit -> Insert Image в Jupyter или с помощью кнопки "Вставить изображение" в Colab. Следите за размером итогового файла.

Фотографии принимаются только в хорошем качестве, **следите за освещением и почерком**. На фотографиях также указывайте номера задач.

-----

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

from statsmodels.sandbox.stats.multicomp import multipletests
from tqdm.notebook import tqdm

%matplotlib inline


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#Теоретическая часть

## p-value (основной поток)

###Задача 1. 
 Для всех пунктов задач 3 и 4 прошлого домашнего задания выпишите формулу для p-value  $p(t) = <...>$  в виде кода на `scipy`, где $t$ - реализация статистики вашего критерия, т.е. $t = T(x), x$ &mdash; реализация выборки, $T(X)$ &mdash; статистика. 

Для задачи 4 также посчитайте численные значения для обоих случаев p-value для данного в задаче существа. Какие гипотезы отклоняются? 

Вычисления можно выполнить в Питоне по приведенным вами формулам. 

## Множественная проверка гипотез (основной поток)


###Задача 2.
 Пусть $X_1, ...,  X_n$ --- выборка из неизвестного распределения $\mathsf{P}$. Для проверки гипотезы $\mathsf{H}_0\ vs.\ \mathsf{H}_1$ было решено использовать три различных критерия. Соответствующие p-value равны 0.00001, 0.7361, 0.0482. Какое должно быть принято решение об отвержении гипотезы $\mathsf{H}_0$ на уровне значимости 0.05? *Подсказка: используйте любой подходящий метод МПГ, далее делайте вывод об отвержении/не отвержении, поясните свой вывод.*




**Задача 1**

In [ ]:
print(f'p-value песик vs единорог(в): {round(sps.gamma(a = 2, scale = 7/6).sf(6.66), 3)}')
print(f'p-value единорог vs песик(г): {round(sps.gamma(a = 3, scale = 44/5).cdf(6.66), 3)}')

p-value песик vs единорог(в): 0.022
p-value единорог vs песик(г): 0.041


**В обоих случаях уровень значимости $\alpha = 0.05$, оба p-value меньше него $\Rightarrow$ животное с ростом x=6.66 не является ни песиком, ни единорогом.** 

**Заметим, что во втором случае p-value ближе к $\alpha$, то есть мы более уверены в том что животное не песик, чем в том что не единорог**

**Задача 2**

**Уже проведены некоторые исследования, значит этап отсеивания признаков прошел. То есть сейчас нужно реализовать метод контроля FWER. 1 гипотеза и три критерия для нее $\Leftrightarrow$ 3 зависимые выборки(на самом деле одна и та же - начальная) и для каждой свой критерий.**

**Воспользуемся методом Холма, c скорректированными p-value:**
$$\tilde{p_j} = (m - j + 1)\cdot p_j,\quad j \in [1,3]$$
$\alpha = 0.05, m = 3$

In [ ]:
def p_values_correct(p_v, m = None, a = 0.05):
  if m is None:
    m = p_v.shape[0]
  p_corr = np.array([(m - j)*p for j,p in enumerate(p_v)]) #отсчет j с 0: m - (j + 1) + 1 = m - j
  ind = 0
  p_first = 0
  for i,elem in enumerate(p_corr): #ищем первую неотверженную гипотезу => cоответствующее значение p_value и индекс
    if elem > a:
      ind = i
      p_first = elem
      break
  for i,p in enumerate(p_corr): #если H_ind не отверглась, то дальше не отвергаем
    if i > ind:
      p_corr[i] = min(1, max(p_first, p))
  return p_corr

alfa = 0.05
m = 3
p_v = np.array([ 0.00001, 0.7361, 0.0482])
p_corr = p_values_correct(p_v, 3, alfa)
print(*p_corr < alfa) 

True False False


**То есть по первому критерию гипотезу стоит отвергнуть, по двум другим - нет: результат статистически незначим.**

# Практическая часть

## Множественная проверка гипотез


###Задача 3 (все потоки).
Проведены эксперименты для оценки эффективности нескольких препаратов для снижения послеоперационной тошноты. Результаты экспериментов приведены в таблице ниже. При проведении эксперимента пациенты делились на группы случайным образом.

    
                            Количество пациентов  Количество случаев возникновения тошноты

		Плацебо                80                    45 

		Хлорпромазин           75                    26 
    
		Дименгидринат          85                    52 
    
		Пентобарбитал (100 мг) 67                    35 
    
		Пентобарбитал (150 мг) 85                    37 
    

 Проведите сравнение каждого препарата по эффективности по отношению к плацебо c использованием критерия Вальда (см. лекцию 5 и последующие). Какие методы МПГ, контролирующие FWER и FDR, можно использовать в данной задаче? Какие ответы можно получить для этих методов? В каждом случае приведите значения статистики критерия Вальда, p-value и скорректированные p-value. Поясните смысл p-value и множественной проверки гипотез в данной задаче. Оформите решение структурированно. 

**В приципе выборки могут быть зависимыми: если 1 человек участвовал в нескольких исследованиях, что нелогично конечно, но в приципе возможно. Для контроля FWER выберем метод Холма, так как он наиболее мощный, если нет информации о зависимостях. Аналогично, для FDR выберем метод Бенджамини-Иекутиели.**

In [ ]:
data = pd.DataFrame([["Плацебо", 80, 45],
    ["Хлорпромазин", 75, 26],
    ["Дименгидринат", 85, 52],
    ["Пентобарбитал (100 мг)", 67, 35],
    ["Пентобарбитал (150 мг)", 85, 37]])

data.columns = ["Название", "Количество пациентов", "Количество случаев возникновения тошноты"]

In [ ]:
data 

,Название,Количество пациентов,Количество случаев возникновения тошноты
0,Плацебо,80,45
1,Хлорпромазин,75,26
2,Дименгидринат,85,52
3,Пентобарбитал (100 мг),67,35
4,Пентобарбитал (150 мг),85,37


**Аналогичный пример был разобран на лекции, то что было рассказано примем в этом дз без доказательства**

**Исследуемая группа - под действием препарата**:

$$ \text{Хлорпромазин: }X^1 = (X_1^1, \cdots ,X_{75}^1), X_i^1 \thicksim Bern(p_1)$$
$$\text{Дименгидринат: } X^2 = (X_1^2, \cdots ,X_{85}^2), X_i^2 \thicksim Bern(p_2)$$
$$ \text{Пентобарбитал (100 мг): } X^3 = (X_1^3, \cdots ,X_{67}^2), X_i^3 \thicksim Bern(p_3)$$
$$ \text{Пентобарбитал (150 мг): }X^4 = (X_1^4, \cdots ,X_{85}^2), X_i^4 \thicksim Bern(p_4)$$
**Контроль:**
$$Y = (Y_1, \cdots, Y_{80}), Y_i \thicksim Bern(p_c) $$

 
**Для каждого препарата:**
$$\text{Статистика критерия Вальда: }W_i = W(X^i,Y) = \frac{\hat{p_i} - \hat{p_c}}{\hat{\sigma}} \rightarrow \mathcal{N}(0,1)$$
где $$\hat{p_i} = 1- \bar{X^i}\quad \hat{p_c} = 1 -\bar{Y}$$
так как вероятность того что препарат работает это количество случаев, когда не возникло тошноты
$$\hat{\sigma} = \sqrt{\frac{\hat{p_i} (1- \hat{p_i})}{n} + \frac{\hat{p_c} (1- \hat{p_c})}{m(=80)}}$$
$\quad$
**Гипотезы:**
$$H_0^i: p_i = p_c \text{ - эффекта нет}$$
$$H_1^i: p_i > p_c \text{ - препарат эффективнее плацебо}$$
$w_i = W_i(x)$
$$p-value: p(w) = P(W(X^i, Y) > w_i)$$

In [ ]:
p_s = 1 - np.array(data["Количество случаев возникновения тошноты"]/data["Количество пациентов"])
n = np.array(data["Количество пациентов"][1:])
p_c, p_s = p_s[0], p_s[1:]

sigma_s = np.sqrt((p_s*(1-p_s)/n)+ (p_c*(1-p_c)/80))
walds = (p_s - p_c)/sigma_s
cr_walds = {key: round(walds[i],3) for i,key in enumerate(data["Название"][1:])}  #вспомогательный массив для красивого вывода 
print(f'Значения статистики критерия Вальда: {cr_walds}')

p_val = sps.norm().sf(walds)
p_val_s = {key: round(p_val[i],3) for i,key in enumerate(data["Название"][1:])} 
print(f'p-values: {p_val_s}')

Значения статистики критерия Вальда: {'Хлорпромазин': 2.764, 'Дименгидринат': -0.643, 'Пентобарбитал (100 мг)': 0.486, 'Пентобарбитал (150 мг)': 1.647}
p-values: {'Хлорпромазин': 0.003, 'Дименгидринат': 0.74, 'Пентобарбитал (100 мг)': 0.313, 'Пентобарбитал (150 мг)': 0.05}


**По методу контроля FDR Бенджамини-Иекутиели:**

In [ ]:
reject, p_val_by = multipletests(p_val, method = 'fdr_by', alpha = 0.1)[:2]
print(*reject)

True False False False


**Получаем, что при заданном $\alpha = 0.05$ стоит отвергнуть гипотезу о  неэффективности Хлорпромазина, остальных лекарств - нет.**

**По методу Холма контроля FWER:**

In [ ]:
p_corr = p_values_correct(p_val, m = 4)
p_corr_s ={key: round(p_corr[i],3) for i,key in enumerate(data["Название"][1:])} 
print(f'Cкорректированные p_value: {p_corr_s}')

Cкорректированные p_value: {'Хлорпромазин': 0.011, 'Дименгидринат': 2.22, 'Пентобарбитал (100 мг)': 1.0, 'Пентобарбитал (150 мг)': 1.0}


**Сравнивая с $\alpha = 0.05$ получаем, что скорее всего ни одно из лекарств, кроме Хлорпромазина, не эффективнее плацебо** 

**Вывод:**

**В данной задаче значения статистики критерия Вальда для различных выборок весьма различаются, при этом сам критерий имеет вид:**
$$S = \{x^i\in X^i: W(x^i,y) > z_{1-\alpha} \}$$ 
$$\alpha = 0.05 \Rightarrow z_{1-\alpha} = 1.64$$
**Из полученных значений $W(x^i,y)$ не превосходит это $z_{1-\alpha}$ только для Пентобарбитала (150 мг) и Хлопромазина, отстальные результаты статистически не значимы. При переходе к p-value получили нормированную метрику уверенности в гипотезе, p-value не привосходит $\alpha$ (что следовало ожидать) для тех же лекарств.** 

**Однако этому результату нельзя доверять и требуется провести контроль ошибок путем МПГ. Анализируя FDR и FWER получаем, что скорее всего, наиболее эффективным препаратом против тошноты по отношению к плацебо будет только Хлопромазин. При этом, если бы не применяли МПГ, то при $\alpha = 0.05$ могли бы отвергнуть нулевую гипотезу и для Пентобарбитала(150 мг)**